In [1]:
# pip3 install mariadb
# pip3 install pandas
# pip3 install numpy  
# sudo apt-get install libatlas-base-dev
# pip3 install matplotlib
# pip3 install ipympl
# pip3 install ipywidgets

import os
import sys
import mariadb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, interactive_output, fixed, interact_manual, IntSlider, IntRangeSlider, HBox
import ipywidgets as widgets

In [24]:
username = os.environ.get("db_user")
password = os.environ.get("db_pwd")

def update_db_name(change):
    #print(change.new)
    global db_name
    db_name = change.new
    
list_db_name = widgets.RadioButtons(
        options=['reformer', 'catalyst', 'Reformer_BW', 'Reformer_SE'],
        description='Select Database:',
        disabled=False
        )
db_name = list_db_name.value
list_db_name.observe(update_db_name, 'value')
display(list_db_name)

RadioButtons(description='Select Database:', options=('reformer', 'catalyst', 'Reformer_BW', 'Reformer_SE'), v…

In [27]:
print(db_name)

Reformer_BW


In [28]:
try:
    # Connect to MariaDB Platform
    conn = mariadb.connect(
        user=username,
        password=password,
        host="localhost",
        port=3306,
        database=db_name,
        autocommit=True
    )

    # Get Cursor for tx
    cur = conn.cursor(named_tuple=False)
    #print(conn)
    print(cur)
    print(f"Connected to {db_name}@localhost ")
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

Connected to Reformer_BW@localhost 


In [32]:
%matplotlib widget

output = widgets.Output()
with output:
    fig_TC, ax_TC = plt.subplots(constrained_layout=True, figsize=(10, 7))
fig_TC.canvas.toolbar_position = 'left'

cur.execute('SHOW tables')
table_dropdown = widgets.Dropdown(options=[u for i in cur.fetchall()[:] for u in i][::-1], description='Database')
cur.execute(f"SELECT * FROM {table_dropdown.value}")
cur.fetchall()
time_slider = widgets.IntRangeSlider(value=[0, cur.rowcount], min=0, max=cur.rowcount, step=1200, description='Time Range',continuous_update=False)

def update_time_range(*args):
    cur.execute(f"SELECT * FROM {table_dropdown.value}")
    cur.fetchall()
    time_slider.max = cur.rowcount
table_dropdown.observe(update_time_range, 'value')

def plot_TC(Table_name, Time):    
    cur.execute(f"SELECT * FROM {Table_name}")
    df = pd.DataFrame(cur.fetchall(), columns=[entry[0] for entry in cur.description]).set_index('Id')
    plt.cla()
    #ax2 = ax1.twinx()
    df['TC7'].plot(legend=True, ax=ax_TC, kind='area', color='lightblue')
    df['TC8'].plot(legend=True, ax=ax_TC)
    df['TC9'].plot(legend=True, ax=ax_TC)
    df['TC10'].plot(legend=True, ax=ax_TC)
    df['EVA_out'].plot(legend=True, ax=ax_TC,
                       title=f'TC_{Table_name}', 
                       xlabel='Time[s]', 
                       ylabel='Temp[oC]', 
                       grid=True,  
                       secondary_y=False, 
                       color='purple', 
                       xlim=Time, 
                       xticks=range(Time[0],Time[1],1200),
                       ylim=(0,1000), 
                       yticks=range(0,1000,50),
                      )
    #ax1.set_ylabel('Temp_SR [oC]', color='g')
    #ax2.set_ylabel('Temp_BR [oC]', color='b')
    #fig_TC.canvas.draw()
    #plt.legend(loc='upper right')
    plt.show()
    

plot = interactive_output(plot_TC, {'Table_name':table_dropdown, 'Time':time_slider})
control = HBox([table_dropdown, time_slider])
widgets.VBox([control, plot])

In [31]:
%matplotlib widget

output = widgets.Output()
with output:
    fig_DFM, ax_DFM = plt.subplots(constrained_layout=True, figsize=(10, 5))
fig_DFM.canvas.toolbar_position = 'left'

cur.execute('SHOW tables')
table_dropdown = widgets.Dropdown(options=[u for i in cur.fetchall()[:] for u in i][::-1], description='Database')
cur.execute(f"SELECT * FROM {table_dropdown.value}")
cur.fetchall()
time_slider = widgets.IntRangeSlider(value=[0, cur.rowcount], min=0, max=cur.rowcount, step=1200, description='Time Range',continuous_update=False)

def update_time_range(*args):
    cur.execute(f"SELECT * FROM {table_dropdown.value}")
    cur.fetchall()
    time_slider.max = cur.rowcount
table_dropdown.observe(update_time_range, 'value')

def plot_TC(Table_name, Time):    
    cur.execute(f"SELECT * FROM {Table_name}")
    df = pd.DataFrame(cur.fetchall(), columns=[entry[0] for entry in cur.description]).set_index('Id')
    plt.cla()
    df = pd.DataFrame({
        'DFM_AOG':df['DFM_AOG'],
        'DFM_RichGas':df['DFM_RichGas'],
    }, index=df.index)
    df.plot(legend=True, ax=ax_DFM, kind='area', stacked=True, 
            title=f'TC_{Table_name}',
            xlabel='Time[s]', 
            ylabel='Temp[oC]', 
            grid=True,  
            xlim=Time, 
            xticks=range(Time[0],Time[1],1200),
            ylim=(0,150), 
            yticks=range(0,150,10)
           )
    plt.show()
    

plot = interactive_output(plot_TC, {'Table_name':table_dropdown, 'Time':time_slider})
control = HBox([table_dropdown, time_slider])
widgets.VBox([control, plot])

In [ ]:
conn.close()